In [1]:
import win32com.client
import os
from datetime import datetime, timedelta

# Магия, подключаемся к аутлук
outlook = win32com.client.Dispatch('outlook.application')
mapi = outlook.GetNamespace('MAPI')

In [2]:
# 6 - это папка "Входящие"
# Нас интересует подпапка (subfolder)
inbox = mapi.GetDefaultFolder(6)
subfolder = inbox.Folders['Свид-ва СЭДО']

In [3]:
# Выгребаем все сообщения (lazy метод, запроса не осуществляется)
messages = subfolder.Items

In [4]:
# Фильтр сообщений, в данном случае по дате на последние сутки
received_dt = datetime.now() - timedelta(days=1)
received_dt = received_dt.strftime('%d/%m/%Y %H:%M %p')

messages = messages.Restrict("[ReceivedTime] >= '" + received_dt + "'")

In [5]:
messages

<COMObject Restrict>

In [6]:
base_dict = {
    'PERNR': '00000000',
    'BEGDA': '01011900',
    'ENDDA': '31129999',
    'FORM': '',
    'INSTITUTE': '',
    'DOCUMENTYPE': 'Z1',
    'ZZANZKL': '1',
    'ZZANZEH': 'часы',
    'ZZIDKUR': '00000000',
    'ZZPASRK': '',
    'ZZSTART': '',
    'ZZEND_DATE': '01011900',
    'ZZNRASP': '',
    'ZZPRODA': '',
    'ZZNPROT': '',
    'ZZPROTA': '',
    'ZZNDOCO': 'Number_of_document',
    'ZZPRONA': '01011900',
    'ZZVIDOB': '87002000',
    'ZZINSTI': '',
    'ZZRAZR': '',
    'ZZDOPI': ''
}

res = []

In [7]:
import re
for message in list(messages):
    tmp_dict = dict.copy(base_dict)
    
    # pernr
    pernr = re.search(r'(ID \d{8})', message.body)
    pernr = pernr[0].split()[-1]
    # print(pernr)
    
    # objid
    objid = re.search(r'по курсу .+', message.body)
    objid = re.search(r'\d{8}', objid[0].split()[-1])[0]
    
    # Номер свидетельства
    doc_num = re.search(r'свидетельства номер ".+"', message.body)
    
    # Костыль для вывода содержимого между кавычек
    match = re.search(r'".+"', doc_num[0])
    
    #doc_num = doc_num[0].split()[-1].replace('"', '')
    doc_num = match[0].replace('"', '')
    
    # Даты
    doc_date = re.search(r'от \d{2}.\d{2}.\d{4}', message.body)
    doc_date = doc_date[0].split()[-1].replace('.', '')
    
    
    tmp_dict['PERNR'] = pernr
    tmp_dict['ZZIDKUR'] = objid
    tmp_dict['ZZNDOCO'] = doc_num
    
    # Даты, одно значение в несколько полей
    tmp_dict['BEGDA'] = doc_date
    tmp_dict['ZZEND_DATE'] = doc_date
    tmp_dict['ZZPRONA'] = doc_date
    
    res.append(tmp_dict)
    

In [8]:
import pandas as pd
list_of_dict = pd.DataFrame(res).to_dict(orient='list')
res_df = pd.DataFrame.from_dict(list_of_dict)
res_df

""


In [137]:
# выгрузка в эксель
res_df.to_excel('Свидетельства.xlsx', sheet_name='res', index=False)